# Strategies in Particle Distributions

The behavior and properties of aerosols are significantly influenced by the characteristics of the particles they contain, including their mass, size, and concentration. To accurately model these aspects, we employ different **particle distribution strategies** that allow us to represent and manipulate particle data effectively.

### The Strategy Pattern

Utilizing the **[Strategy Pattern](https://en.wikipedia.org/wiki/Strategy_pattern)**, we can define a family of algorithms, encapsulate each one, and make them interchangeable. This pattern lets the algorithm vary independently from clients that use it, providing flexibility in the particle representation.

### Key Components:

**`ParticleStrategy`**: An abstract base class that outlines the common interface for calculating particle mass, radius, and total mass.

- **MovingBins**: Are a collection of strategies that move the bins in a distribution, when mass is added or removed from the system. This is useful for simulating the movement of particles in a system.
    - **`MassBaseMovingBin`**: A concrete strategy for particles represented by their mass distribution.
    - **`RadiiBasedMovingBin`**: Focuses on particles defined by their radius (size distribution).
    - **`SpeciatedMassMovingBin`**: Tailored for particles with a distribution across different species, each potentially having distinct densities.

The `distribution` is the x-axis of a particle distribution, so it can be mass, radius, or species-masses. The `concentration` are the y-axis of the distribution, currently all number-concentration. It is these two parameters that define the particle distribution, and they change depending on the strategy.

Additional strategies can be added to the system `ParticleStrategy`, such as a PDF representation of the distribution, or fixed bins that do not move.

By adopting these strategies, we can easily switch between different methods of representing particle distributions, enabling more nuanced simulations and analyses of aerosol behavior.


In [13]:
# Example demonstrating the usage of particle distribution strategies

# necessary imports
import numpy as np
from particula.next.particle import (
    particle_strategy_factory,
    Particle,
    MassBasedMovingBin,
)
from particula.next.particle_activity import particle_activity_strategy_factory
from particula.next.surface import surface_strategy_factory

# Creating particle distribution examples
mass_distribution = np.array([100, 200, 300], dtype=np.float_)
density = 2.5
concentration = np.array([10, 20, 30], dtype=np.float_)

# Instantiate a particle strategy
mass_based_strategy = MassBasedMovingBin()

# Create a Particle instance using the MassBasedStrategy
particle = Particle(strategy=mass_based_strategy,
                    activity=particle_activity_strategy_factory(),
                    surface=surface_strategy_factory(),
                    distribution=mass_distribution,
                    density=density,
                    concentration=concentration)

# Accessing calculated properties
print("Mass of particles:", particle.get_mass())
print("Radius of particles:", particle.get_radius())
print("Total mass of the particle distribution:", particle.get_total_mass())

Mass of particles: [100. 200. 300.]
Radius of particles: [2.12156884 2.67300924 3.05983174]
Total mass of the particle distribution: 14000.0


## Strategy Factory

The ability to switch between different computational strategies for particle properties (mass, radius, and total mass) based on the particle representation is crucial. This flexibility allows for a more accurate and tailored approach to simulating real-world scenarios. The `create_particle_strategy` function serves as a factory that abstracts the instantiation of different particle strategy classes, enabling dynamic strategy selection based on the type of particle representation required.

### Key Features:

- **Flexible Strategy Selection**: By providing a string identifier (`mass_based_moving_bin`, `number_based_moving_bin`, `speciated_mass_moving_bin`), users can easily switch between different computational strategies for particle properties without altering the core logic of their simulation.
- **Simplification of Complex Decisions**: The factory hides the complexity of strategy object creation, making the main application code cleaner and more maintainable.
- **Extendibility**: New particle strategies can be added to the factory with minimal changes to the existing codebase.

This factory function is instrumental in enabling a modular and scalable approach to particle system modeling, catering to a wide range of scientific and engineering applications.


In [14]:
# Example: Creating a mass-based particle strategy
mass_based_strategy = particle_strategy_factory("mass_based_moving_bin")

# Now, initialize a Particle instance using the mass-based strategy
# Example particle sizes in meters
particle_distribution = np.array([1e-9, 2e-9, 3e-9], dtype=np.float_)
particle_density = np.float_(1000)  # Example density in kg/m^3
particle_concentration = np.array(
    [1000, 2000, 3000], dtype=np.float_)  # Example concentrations

particle = Particle(
    strategy=mass_based_strategy,
    activity=particle_activity_strategy_factory(),
    surface=surface_strategy_factory(),
    distribution=particle_distribution,
    density=particle_density,
    concentration=particle_concentration
)

# Demonstrate the usage of the Particle instance
print(f"Particle Masses: {particle.get_mass()}")
print(f"Particle Radii: {particle.get_radius()}")
print(f"Total Particle Mass: {particle.get_total_mass()}")

Particle Masses: [1.e-09 2.e-09 3.e-09]
Particle Radii: [6.20350491e-05 7.81592642e-05 8.94700229e-05]
Total Particle Mass: 1.4000000000000001e-05


## Interface for Particle Distributions

Modeling particles in aerosol science requires accommodating a diverse range of particle representations, from mass-based to number-based distributions, among others. The `Particle` class, in conjunction with the Strategy pattern, provides a powerful and flexible framework to abstract these differences behind a consistent interface. This approach enables seamless integration of various particle representations into aerosol models without altering the consuming code's logic.

### Advantages of Strategy-Powered Particle Class

- **Flexibility**: By decoupling the particle properties computations from the representation, the `Particle` class can adapt to various particle models simply by switching strategies.
- **Maintainability**: With the computational details encapsulated in strategy objects, updating or adding new particle representations becomes straightforward, enhancing the system's maintainability.
- **Scalability**: The Strategy pattern allows for the easy introduction of new particle behaviors and properties, accommodating future scientific discoveries or modeling requirements without disrupting existing implementations.

### Implementation Highlights

The `Particle` class achieves this by delegating the computation of mass, radius, and total mass to the strategy object it holds. This object implements the `ParticleStrategy` interface, ensuring that any strategy adheres to a consistent method signature, regardless of the underlying particle representation.

### Example: Interfacing Particle Distributions

Following the theoretical groundwork, we illustrate the practical application of these concepts:


In [15]:
# Instantiate a Particle with a Mass-Based Strategy
mass_based_particle = Particle(
    strategy=mass_based_strategy,
    activity=particle_activity_strategy_factory(),
    surface=surface_strategy_factory(),
    distribution=np.array([10, 20, 30], dtype=np.float_),
    density=np.float_(2.5),
    concentration=np.array([1000, 2000, 3000], dtype=np.float_)
)

# Instantiate a Particle with a Number-Based Strategy
radii_based_strategy = particle_strategy_factory("radii_based_moving_bin")
radii_based_particle = Particle(
    strategy=radii_based_strategy,
    activity=particle_activity_strategy_factory(),
    surface=surface_strategy_factory(),
    distribution=np.array([10, 20, 30], dtype=np.float_),
    density=np.float_(1.5),
    concentration=np.array([500, 100, 200], dtype=np.float_)
)

# Demonstrating how Particle class methods provide a consistent interface
print("Mass-Based Particle Total Mass:", mass_based_particle.get_total_mass())
print("Number-Based Particle Total Mass:", radii_based_particle.get_total_mass())

Mass-Based Particle Total Mass: 140000.0
Number-Based Particle Total Mass: 42097341.558103226



## Conclusion

The key takeaway, is that this `Particle` interface allows for a consistent way to interact with different particle representations, while the strategy pattern allows for the encapsulation of the computational details of each representation. This combination provides a powerful and flexible framework for modeling particles in aerosol science and engineering, enabling the seamless integration of various particle representations into aerosol models without altering the consuming code's logic.

The low coupling between the `Particle` class and the `ParticleStrategy` interface provides a clear separation of concerns, allowing for independent evolution of particle representations and computational strategies.

## Future Work

New strategies can be added, like a super-droplet representation, or single-particle representation. The `Particle` class can be extended to include additional properties and methods, such as particle charge, temperature, and velocity, further enhancing its utility in aerosol science and engineering. Additionally, the `ParticleStrategy` interface can be expanded to include more methods for advanced particle property computations, broadening its scope and applicability.

# Help

In [16]:
help(Particle)

Help on class Particle in module particula.next.particle:

class Particle(builtins.object)
 |  Particle(strategy: particula.next.particle.ParticleStrategy, activity: particula.next.particle_activity.ParticleActivityStrategy, surface: particula.next.surface.SurfaceStrategy, distribution: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], density: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], concentration: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]])
 |  
 |  Represents a particle or a collection of particles, encapsulating the
 |  strategy for calculating mass, radius, and total mass based on a
 |  specified particle distribution, density, and concentration. This class
 |  allows for flexibility in representing particles by delegating computation
 |  to a strategy pattern.
 |  
 |  Attributes:
 |  - strategy (ParticleStrategy): The computation strategy for particle
 |  representations.
 |  - activity (ParticleActivityStrategy): The activity strategy for the
 |  p

In [17]:
help(particle_activity_strategy_factory)

Help on function particle_activity_strategy_factory in module particula.next.particle_activity:

particle_activity_strategy_factory(strategy_type: str = 'mass_ideal', **kwargs: dict)
    Factory function for creating activity strategies. Used for calculating
    activity and partial pressure of species in a mixture of liquids.
    
    Args:
    - strategy_type (str): Type of activity strategy to use. The options are:
        - molar_ideal: Ideal activity based on mole fractions.
        - mass_ideal: Ideal activity based on mass fractions.
        - kappa: Non-ideal activity based on kappa hygroscopic parameter.
    - kwargs: Arguments for the activity strategy.



In [18]:
from particula.next.particle import ParticleStrategy
help(ParticleStrategy)

Help on class ParticleStrategy in module particula.next.particle:

class ParticleStrategy(abc.ABC)
 |  Abstract base class for particle strategy, defining the common
 |  interface for mass, radius, and total mass calculations for different
 |  particle representations.
 |  
 |  Method resolution order:
 |      ParticleStrategy
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add_mass(self, distribution: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], concentration: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], density: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], added_mass: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]]) -> tuple[numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]]]
 |      Adds mass to the distribution of particles based on their distribution,
 |      concentration, and density.
 |      
 |      Parameters:
 |      - distribution (NDArray[np.float_]

In [19]:
help(MassBasedMovingBin)

Help on class MassBasedMovingBin in module particula.next.particle:

class MassBasedMovingBin(ParticleStrategy)
 |  A strategy for particles represented by their mass distribution, and
 |  particle number concentration. Moving the bins when adding mass.
 |  
 |  Method resolution order:
 |      MassBasedMovingBin
 |      ParticleStrategy
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  add_mass(self, distribution: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], concentration: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], density: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], added_mass: numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]]) -> tuple[numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]], numpy.ndarray[typing.Any, numpy.dtype[numpy.float64]]]
 |      Adds mass to the distribution of particles based on their distribution,
 |      concentration, and density.
 |      
 |      Parameters:
 |      - distribution (NDArray